In [2]:
from paho.mqtt import client as mqtt_client
from datetime import datetime
import csv
import json

In [3]:
print(datetime.now())

2024-03-26 00:19:06.619077


In [ ]:
broker = 'broker.emqx.io'
port = 1883
topic = "move_id/AA"
client_id = ''

message_counter = 0  

def append_to_csv(data):
    with open('data_not_normal.csv', mode='a', newline='') as file:
        writer = csv.writer(file)

        # Check if the file is empty and write header if it is
        if file.tell() == 0:
            writer.writerow(['date', 'x', 'y', 'z'])

        writer.writerow(data)


def connect_mqtt() -> mqtt_client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1,client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        global message_counter  
        time = datetime.now()
        data = json.loads(msg.payload.decode())
        
        
        x = float(data['accelerometerSensor']['x'])
        y = float(data['accelerometerSensor']['y'])
        z = float(data['accelerometerSensor']['z'])
        
        
        append_to_csv([time, x, y, z])
        
        print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
        
        message_counter += 1 
        
        if message_counter >= 120:
            print("Received 120 messages, stopping the run.")
            client.disconnect()  
            client.loop_stop()    

    client.subscribe(topic)
    client.on_message = on_message


def run():
    client = connect_mqtt()
    subscribe(client)
    client.loop_forever()


if __name__ == '__main__':
    run()